## 인풋 파일 만들기

In [ ]:
import os
import json
import glob
import matplotlib.pyplot as plt
import cv2
import shutil
import yaml
from sklearn.model_selection import train_test_split

def remove_folder(path):
    if os.path.exists(path):#파일이 있으면
        shutil.rmtree(path) #제거
        print(f'{path} 폴더 제거 완료')
    if not os.path.exists(path):#파일이 없으면
        os.mkdir(path)
        print(f'{path} 폴더 생성 완료')

def make_folder(path):
    if not os.path.exists(path): # 이미지 폴더가 없으면
        os.makedirs(path)# 폴더 생성
        print(f'{path} 폴더 생성 완료')
        
def saving(path, part, png, filename=None, anno=None, width=None, height=None):
    images_path = f'{path}/images/{part}'
    labels_path = f'{path}/labels/{part}'
    
    # 이미지 저장
    src = cv2.imread(png)
    img = cv2.resize(src, dsize=(960, 512), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(images_path+'/'+png.split('/')[-1] ,img)
    
    # 이미지 경로 저장
    if not os.path.exists(f"{path}/{part}.txt"):# 파일이 없으면
        
        f = open(f"{path}/{part}.txt",'w') # 파일 생성
#         print(f'{path}/{part}.txt 생성 완료')
        f.close()
        
    with open(f"{path}/{part}.txt", 'a')as f:#  이어쓰기
        f.write(f"{images_path}/{png.split('/')[-1]}\n")

        for i in anno:
            label = i['attribute'] # 라벨명
            bbox = i['points'] # bbox 좌표 
            x,y,w,h = ((bbox[0][0]+bbox[2][0])/2), ((bbox[0][1]+bbox[2][1])/2),\
            (bbox[2][0] - bbox[0][0]), (bbox[2][1]-bbox[0][1])#

            # 라벨.txt 저장
            if not os.path.exists(f"{labels_path}/{filename}"):# 파일이 없으면
                f = open(f"{labels_path}/{filename}",'w') # 파일 생성
#                 print(f'{labels_path}/{filename} 생성 완료')
                f.close()
            with open(f"{labels_path}/{filename}", 'a') as f: # 이어 쓰기
                    f.writelines(f"{atts[label]} {x} {y} {w} {h}\n")
                

atts = {'car': 0,'pedestrian': 1,'truck': 2,'bus': 3 }

# 만든 데이터를 저장할 폴더 (없으면 만들어줌)
path = '/aiffel/aiffel/yolov7_start/input_file'

# 학습 데이터가 있는 경로
png_path = sorted(glob.glob('/aiffel/aiffel/datasets/final_dataset/images/*.png'))
json_path = sorted(glob.glob('/aiffel/aiffel/datasets/final_dataset/labels/*.json'))


# 만들어둔 input 데이터가 있으면 삭제하고 폴더만 재생성
remove_folder(path)

# train, val, test 각각의 폴더 만들기
make_folder(path+'/images/train')
make_folder(path+'/labels/train')
make_folder(path+'/images/val')
make_folder(path+'/labels/val')
make_folder(path+'/images/test')
make_folder(path+'/labels/test')

# 학습,검증셋 분리
train, val = train_test_split(json_path, test_size=0.2, random_state=2022)

for png, json_ in zip(png_path, json_path):# 데이터 읽어와서
    with open(json_, 'r')as f:
        dict = json.load(f)
    # 파일 이름 +.txt --> 라벨 저장할 txt
    filename = dict['filename'][:-4]+'.txt'
    anno = dict['annotations']
    # 데이터 개수 지정
    if json_ in train: #  train 
        saving(path, 'train', png, filename, anno)
        
    elif json_ in val:# val
        saving(path, 'val', png, filename, anno)
        

# 테스트 데이터가 있는 경로
test_png_path = sorted(glob.glob('/aiffel/aiffel/datasets/final_test/images/*.png'))
test_json_path = sorted(glob.glob('/aiffel/aiffel/datasets/final_test/labels/*.json'))

# 테스트 파일 따로 생성
for png, json_ in zip(test_png_path, test_json_path):# 데이터 읽어와서
    with open(json_, 'r')as f:
        dict = json.load(f)
    # 파일 이름 +.txt --> 라벨 저장할 txt
    filename = dict['filename'][:-4]+'.txt'
    anno = dict['annotations']
    saving(path, 'test', png, filename, anno)

print('데이터 생성 완료')
    
# yaml만들기

atts_dict = {i:j for i,j in enumerate(atts)}

yaml_dict = {'train':path+'/train.txt', 'val':path+'/val.txt', 'test':path+'/test.txt', 'nc':len(atts),'names' :list( atts_dict.values())}

# yaml저장
if not os.path.exists(f"{path}/test.yaml"):# 파일이 없으면
    f = open(f"{path}/test.yaml",'w') # 파일 생성
    f.close()
with open(f"{path}/test.yaml", 'a') as f: # 이어 쓰기
    yaml.dump(yaml_dict,f, sort_keys = False)
    print('yaml생성 완료')
       

## val,test json 만들기

In [ ]:
import json
from glob import glob

def mk_json(path, save_dir):
    jsons = sorted(glob(path))
    new_json = {}
    imgs, anno, ct= [],[],[]
    category_ids = {'car':0,'pedestrian':1, 'truck':2, 'bus':3}
    for idx,i in enumerate(jsons):
        with open(i,'r')as f:
            info = json.load(f)
            
        imgs.append({"file_name": info['filename'],"height": 512,"width": 960,"id": info['filename'][:-4]})
        
        for idx_a, j in enumerate(info['annotations']):
            area = (j['points'][2][0] - j['points'][0][0])*960*(j['points'][2][1] - j['points'][0][1])*512
            bbox = (j['points'][2][0]+j['points'][0][0])/2*960, (j['points'][2][1]+j['points'][0][1])/2*512,(j['points'][2][0]-j['points'][0][0])*960,(j['points'][2][1]-j['points'][0][1])*512
            category_id = category_ids[j['attribute']]
#             j['points'][3][0]*960, j['points'][3][1]*512
            anno.append({'image_id':info['filename'][:-4],'iscrowd':0, 'area':area, 'bbox':bbox, 'category_id':category_id, 'id':idx_a})
            
    new_json['images'] = imgs
    new_json['annotations'] = anno
    new_json['categories'] = [{'id': 0, 'name': 'car'},{'id': 1, 'name': 'pedestrian'},
                              {'id': 2, 'name': 'truck'}, {'id': 3, 'name': 'bus'} ]
    
    with open(save_dir,'w')as f:
        json.dump(new_json,f, sort_keys = False)
        print('json 생성 완료')
    
val_path = '/aiffel/aiffel/datasets/final_dataset/labels/*.json'
val_save = '/aiffel/aiffel/yolov7_start/input_file/val.json'
test_path = '/aiffel/aiffel/datasets/final_test/labels/*.json'
test_save = '/aiffel/aiffel/yolov7_start/input_file/test.json'

mk_json(val_path, val_save)
mk_json(test_path, test_save)